In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist= input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [11]:
n_nodes_hl_mean=784
n_nodes_hl_log_cov=784
n_nodes_out = 784

In [12]:
x=tf.placeholder('float',[None,784])
y=tf.placeholder('float')

In [13]:
def neural(data):
    hidden_1_mean={'weights':tf.Variable(tf.random_normal([784, n_nodes_hl_mean])),
    'biases':tf.Variable(tf.random_normal([n_nodes_hl_mean]))}
    hidden_1_log_cov={'weights':tf.Variable(tf.random_normal([784, n_nodes_hl_log_cov])),
    'biases':tf.Variable(tf.random_normal([n_nodes_hl_log_cov]))}
    
    output_layer={'weights':tf.Variable(tf.random_normal([n_nodes_out, 784])),
    'biases':tf.Variable(tf.random_normal([784]))}

    l1= tf.add(tf.matmul(data, hidden_1_mean['weights']), hidden_1_mean['biases'])
    l1= tf.nn.leaky_relu(l1)
    l2= tf.add(tf.matmul(data, hidden_1_log_cov['weights']), hidden_1_log_cov['biases'])
    l2= tf.nn.leaky_relu(l2)
    l3= l1 + (tf.sqrt(tf.exp(l2)) * np.random.normal(0,1))
    l3= tf.nn.leaky_relu(l3)
    output= tf.matmul(l3, output_layer['weights'])+output_layer['biases']
    return [output,l2,l1]

In [17]:
def train(x):
    [prediction,l2,l1]=neural(x)
    img_loss = tf.reduce_sum(tf.squared_difference(x, prediction), 1)
    latent_loss = -0.5 * tf.reduce_sum((1.0 + l2 - tf.square(l1) - tf.exp(l2)), 1)
    cost = tf.reduce_mean(img_loss + latent_loss)
    optimizer=tf.train.AdamOptimizer().minimize(cost)
    hm_epochs=50

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss=0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size, shuffle=True)
                _,c=sess.run([optimizer,cost],feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:',epoch_loss)
        

In [18]:
train(x)

InternalError: Failed to create session.